In [ ]:
import numpy as np
import pandas as pd
import wbdata
import random
import re
from sklearn.preprocessing import Imputer

columns = ['Country', 'Year', 'Status', 'Life_Expectancy', 
           'Adult_Mortality', 'Infant_Deaths', 'Alcohol', 
           'Percentage_Expenditure', 'Measles', 'BMI_5-19_years',
           'BMI_18+_years','Under-Five_Deaths', 'Polio', 
           'Total_Expenditure', 'Diphtheria', 'GDP', 
           'Population','Thinness_10-19_years', 
           'Thinness_5-9_years','Income_Composition_Of_Resources', 'Schooling']
main_data = pd.DataFrame(columns=columns)

len(main_data.columns)

## COMMON REGEX

In [ ]:
regex_double_nums = re.compile(r'^[\d]+([\d]+|.[\d]+) [\d]+([\d]+|.[\d]+)$')

Sometimes two integers popup in the same cell so we have to filter out those values and select only the first value.

## Population Data

In [ ]:
wbdata.search_indicators('Population')

#set up the indicator I want (just build up the dict if you want more than one)
indicators = {'SP.POP.TOTL':'Population'}

#grab indicators above for all countries and load into data frame
pop_df_in = wbdata.get_dataframe(indicators, convert_date=False)

pop_df_in.to_csv("Data/New/Population/Population_Internet.csv", encoding='utf-8')

pop_df_get = pd.read_csv("Data/New/Population/Population_Internet.csv")
pop_df = pop_df_get.rename(index = str, columns={'date':'Year', 'country':'Country'})
len(pop_df['Country'].unique())

# Removing continents and other trivial rows
pop_df.drop(pop_df.index[:2773], inplace=True)
len(pop_df['Country'].unique())

# Year: 2000 - 2014
pop_df = pop_df[(lambda x: x['Year'].isin(range(2000,2015)))]

pop_df.reset_index(drop=True, inplace=True)

pop_df.head()

# Fixing Missing Values
pop_df.at[915, 'Population'] = 5054634
pop_df.at[916, 'Population'] = 4945529
pop_df.at[917, 'Population'] = 4840901

main_data[['Country', 'Year', 'Population']] = pop_df[['Country', 'Year', 'Population']]

main_data['Population'].isnull().sum()
#main_data.info()

#main_data[main_data['Country'] == "Vietnam"]

## GDP

In [ ]:
#set up the indicator I want (just build up the dict if you want more than one)
indicators = {'NY.GDP.PCAP.CD':'GDP'}

#grab indicators above for countires above and load into data frame
gdp_df = wbdata.get_dataframe(indicators, convert_date=False)

gdp_df.to_csv("Data/New/GDP/GDP_Internet.csv", encoding='utf-8')
gdp_data_get = pd.read_csv("Data/New/GDP/GDP_Internet.csv")
gdp_data = gdp_data_get.rename(index = str, columns={'date':'Year', 'country':'Country'})
#gdp_data[gdp_data['Country'] == "South Sudan"]

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(gdp_data[gdp_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            gdp_other_data = gdp_data[gdp_data['Country'] == main_data['Country'][val]][gdp_data['Year'] == main_data['Year'][val]]['GDP'].values[0]
            print(gdp_other_data, "pass: 1")
            main_data.at[val, 'GDP'] = gdp_other_data
        except IndexError:
            pass

main_data[main_data['GDP'].isnull()]['Country'].unique()

#main_data[main_data['GDP'].isnull()][main_data['Country'] == "Afghanistan"][['Country','Year','GDP']]

main_data[main_data['Country'] == "South Sudan"][['Country', 'Year', 'GDP']]
len(main_data['Country'].unique())

## Total Expenditure

In [ ]:
totalex = pd.read_csv("Data/New/Total_Expenditure/data_internet.csv")

count = 0
totalex_data = pd.DataFrame(columns=['Country', 'Year', 'Total_Expenditure'])
for i in range(0, len(totalex['Country'].unique())):
    for j in range(1, len(totalex.columns)):
        #print(totalex.loc[i,'Country'], end='')
        totalex_data.at[count, 'Country'] = totalex.loc[i,'Country']
        totalex_data.at[count, 'Year'] = totalex.columns[j]
       # print(totalex.columns[j])
        totalex_data.at[count, 'Total_Expenditure'] = totalex[totalex['Country'] == totalex['Country'][i]].T[1:].loc[totalex.columns[j]].values[0]
        count+=1

# Dropping "South Sudan" - Many missing values
# totalex_data.drop(totalex_data[totalex_data['Country'] == "South Sudan"].index, inplace=True)

totalex_data['Year'] = totalex_data['Year'].astype(np.int64)
#totalex_data.info()

totalex_data.to_pickle("Data/New/Total_Expenditure/totalex_pickle_data.pkl")

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(totalex_data[totalex_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            totalex_other_data = totalex_data[totalex_data['Country'] == main_data['Country'][val]][totalex_data['Year'] == main_data['Year'][val]]['Total_Expenditure'].values[0]
            print(totalex_other_data, "pass: 1")
            main_data.at[val, 'Total_Expenditure'] = totalex_other_data
        except IndexError:
            pass
    else:
        try:
            totalex_other_data = totalex_data[totalex_data['Country'].str.startswith(main_data['Country'][val])][totalex_data['Year'] == main_data['Year'][val]]['Total_Expenditure'].values[0]
            print(totalex_other_data, "pass: 2")
            main_data.at[val, 'Total_Expenditure'] = totalex_other_data
        except IndexError:
            try:
                totalex_other_data = totalex_data[totalex_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][totalex_data['Year'] == main_data['Year'][val]]['Total_Expenditure'].values[0]
                print(totalex_other_data, "pass: 3")
                main_data.at[val, 'Total_Expenditure'] = totalex_other_data
            except IndexError:
                pass
        

main_data[main_data['Country'].isnull()]['Country'].unique()

totalex_data[totalex_data['Country'].str.startswith("Zimbabwe")]

len(main_data[main_data['Total_Expenditure'].isnull()])

## MEASLES

In [ ]:
measles = pd.read_csv("Data/New/Measles/Measles_Internet.csv")
measles.head()

count = 0
measles_data = pd.DataFrame(columns=['Country', 'Year', 'Measles'])
for i in range(0, len(measles['Country'].unique())):
    for j in range(1, len(measles.columns)):
        #print(measles.loc[i,'Country'], end='')
        measles_data.at[count, 'Country'] = measles.loc[i,'Country']
        measles_data.at[count, 'Year'] = measles.columns[j]
       # print(measles.columns[j])
        measles_data.at[count, 'Measles'] = measles[measles['Country'] == measles['Country'][i]].T[1:].loc[measles.columns[j]].values[0]
        count+=1

measles_data['Year'] = measles_data['Year'].astype(np.int64)

measles_data.to_pickle("Data/New/Measles/measles_pickle_data.pkl")

#measles_data.tail()

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(measles_data[measles_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            measles_other_data = measles_data[measles_data['Country'] == main_data['Country'][val]][measles_data['Year'] == main_data['Year'][val]]['Measles'].values[0]
            print(measles_other_data, "pass: 1")
            main_data.at[val, 'Measles'] = measles_other_data
        except IndexError:
            pass
    else:
        try:
            measles_other_data = measles_data[measles_data['Country'].str.startswith(main_data['Country'][val])][measles_data['Year'] == main_data['Year'][val]]['Measles'].values[0]
            print(measles_other_data, "pass: 2")
            main_data.at[val, 'Measles'] = measles_other_data
        except IndexError:
            try:
                measles_other_data = measles_data[measles_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][measles_data['Year'] == main_data['Year'][val]]['Measles'].values[0]
                print(measles_other_data, "pass: 3")
                main_data.at[val, 'Measles'] = measles_other_data
            except IndexError:
                pass
        

## Income composition of resources

In [ ]:
icor = pd.read_csv("Data/New/Income_Composition_Of_Resources/ICOR_Internet.csv")

count = 0
icor_data = pd.DataFrame(columns=['Country', 'Year', 'Income_Composition_Of_Resources'])
for i in range(0, len(icor['Country'].unique())):
    for j in range(1, len(icor.columns)):
        #print(icor.loc[i,'Country'], end='')
        # Space before a word by default in the dataset. Max_Split = 1 
        icor_data.at[count, 'Country'] = icor.loc[i,'Country'].split(' ', 1)[1]
        icor_data.at[count, 'Year'] = icor.columns[j]
        #print(icor.columns[j])
        icor_data.at[count, 'Income_Composition_Of_Resources'] = icor[icor['Country'] == icor['Country'][i]].T[1:].loc[icor.columns[j]].values[0]
        count+=1

icor_data['Year'] = icor_data['Year'].astype(np.int64)
#icor_data['Income_Composition_Of_Resources'] = icor_data['Income_Composition_Of_Resources'].astype(np.float64)

#icor_data.to_pickle("Data/New/Income_Composition_Of_Resources/ICOR_Internet.pkl")

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(icor_data[icor_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            icor_other_data = icor_data[icor_data['Country'] == main_data['Country'][val]][icor_data['Year'] == main_data['Year'][val]]['Income_Composition_Of_Resources'].values[0]
            print(icor_other_data, "pass: 1")
            main_data.at[val, 'Income_Composition_Of_Resources'] = icor_other_data
        except IndexError:
            pass
    else:
        try:
            icor_other_data = icor_data[icor_data['Country'].str.startswith(main_data['Country'][val])][icor_data['Year'] == main_data['Year'][val]]['Income_Composition_Of_Resources'].values[0]
            print(icor_other_data, "pass: 2")
            main_data.at[val, 'Income_Composition_Of_Resources'] = icor_other_data
        except IndexError:
            try:
                icor_other_data = icor_data[icor_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][icor_data['Year'] == main_data['Year'][val]]['Income_Composition_Of_Resources'].values[0]
                print(icor_other_data, "pass: 3")
                main_data.at[val, 'Income_Composition_Of_Resources'] = icor_other_data
            except IndexError:
                pass
            

#icor_data[icor_data['Country'] == "Zambia"]

#main_data[['Country','Year','Income_Composition_Of_Resources']]

## Diptheria

In [ ]:
diptheria = pd.read_csv("Data/New/Diptheria/Dip_Internet.csv")

count = 0
diptheria_data = pd.DataFrame(columns=['Country', 'Year', 'Diphtheria'])
for i in range(0, len(diptheria['Country'].unique())):
    for j in range(1, len(diptheria.columns)):
        #print(diptheria.loc[i,'Country'], end='')
        diptheria_data.at[count, 'Country'] = diptheria.loc[i,'Country']
        diptheria_data.at[count, 'Year'] = diptheria.columns[j]
        diptheria_data.at[count, 'Diphtheria'] = diptheria[diptheria['Country'] == diptheria['Country'][i]].T[1:].loc[diptheria.columns[j]].values[0]
        count+=1

diptheria_data['Year'] = diptheria_data['Year'].astype(np.int64)
diptheria_data['Diphtheria'] = diptheria_data['Diphtheria'].astype(np.float64)

diptheria_data.to_pickle("Data/New/Diptheria/Dip_Internet.pkl")

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(diptheria_data[diptheria_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            diptheria_other_data = diptheria_data[diptheria_data['Country'] == main_data['Country'][val]][diptheria_data['Year'] == main_data['Year'][val]]['Diphtheria'].values[0]
            print(diptheria_other_data, "pass: 1")
            main_data.at[val, 'Diphtheria'] = diptheria_other_data
        except IndexError:
            pass
    else:
        try:
            diptheria_other_data = diptheria_data[diptheria_data['Country'].str.startswith(main_data['Country'][val])][diptheria_data['Year'] == main_data['Year'][val]]['Diphtheria'].values[0]
            print(diptheria_other_data, "pass: 2")
            main_data.at[val, 'Diphtheria'] = diptheria_other_data
        except IndexError:
            try:
                diptheria_other_data = diptheria_data[diptheria_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][diptheria_data['Year'] == main_data['Year'][val]]['Diphtheria'].values[0]
                print(diptheria_other_data, "pass: 3")
                main_data.at[val, 'Diphtheria'] = diptheria_other_data
            except IndexError:
                pass
            

#main_data[['Country','Year','Diphtheria']]
#main_data.iloc[[3151]]

## Thinness (10-19 Years)

In [ ]:
thin_10_19 = pd.read_csv("Data/New/Thinness_10_19/thin_9_10_internet.csv")
thin_10_19

# Rename empty column name
thin_10_19.rename(columns = {thin_10_19.columns[0]: "Country"}, inplace=True)

count = 0
thin_10_19_data = pd.DataFrame(columns=['Country', 'Year', 'Thinness_10-19_years'])
for i in range(1, len(thin_10_19['Country'].unique())):
    j = 1
    while j < len(thin_10_19.columns):
        #print(thin_10_19.loc[i,'Country'], end=' ')
        thin_10_19_data.at[count, 'Country'] = thin_10_19.loc[i,'Country']
        thin_10_19_data.at[count, 'Year'] = thin_10_19.columns[j]
        #print(thin_10_19.columns[j], j)
        thin_10_19_data.at[count, 'Thinness_10-19_years'] = thin_10_19[thin_10_19['Country'] == thin_10_19['Country'][i]].T[1:].loc[thin_10_19.columns[j]].values[0].split(' [')[0]
        count+=1
        j += 3
        

#thin_10_19_data[thin_10_19_data['Country'] == "Uganda"]

thin_10_19_data['Year'] = thin_10_19_data['Year'].astype(np.int64)
#thin_10_19_data['Thinness_10-19_years'] = thin_10_19_data['Thinness_10-19_years'].astype(np.float64)

thin_10_19_data.to_pickle("Data/New/Thinness_10_19/Thinness_10_19.pkl")

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(thin_10_19_data[thin_10_19_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            thin_10_19_other_data = thin_10_19_data[thin_10_19_data['Country'] == main_data['Country'][val]][thin_10_19_data['Year'] == main_data['Year'][val]]['Thinness_10-19_years'].values[0]
            print(thin_10_19_other_data, "pass: 1")
            main_data.at[val, 'Thinness_10-19_years'] = thin_10_19_other_data
        except IndexError:
            pass
    else:
        try:
            thin_10_19_other_data = thin_10_19_data[thin_10_19_data['Country'].str.startswith(main_data['Country'][val])][thin_10_19_data['Year'] == main_data['Year'][val]]['Thinness_10-19_years'].values[0]
            print(thin_10_19_other_data, "pass: 2")
            main_data.at[val, 'Thinness_10-19_years'] = thin_10_19_other_data
        except IndexError:
            try:
                thin_10_19_other_data = thin_10_19_data[thin_10_19_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][thin_10_19_data['Year'] == main_data['Year'][val]]['Thinness_10-19_years'].values[0]
                print(thin_10_19_other_data, "pass: 3")
                main_data.at[val, 'Thinness_10-19_years'] = thin_10_19_other_data
            except IndexError:
                pass

main_data[['Country','Year','Thinness_10-19_years']].isnull().sum()

thin_5_9 = pd.read_csv("Data/New/Thinness_5_9/thin_5_9_internet.csv")

# Rename empty column name
thin_5_9.rename(columns = {thin_5_9.columns[0]: "Country"}, inplace=True)

count = 0
thin_5_9_data = pd.DataFrame(columns=['Country', 'Year', 'Thinness_5-9_years'])
for i in range(1, len(thin_5_9['Country'].unique())):
    j = 1
    while j < len(thin_5_9.columns):
        #print(thin_5_9.loc[i,'Country'], end=' ')
        thin_5_9_data.at[count, 'Country'] = thin_5_9.loc[i,'Country']
        thin_5_9_data.at[count, 'Year'] = thin_5_9.columns[j]
        #print(thin_5_9.columns[j], j)
        thin_5_9_data.at[count, 'Thinness_5-9_years'] = thin_5_9[thin_5_9['Country'] == thin_5_9['Country'][i]].T[1:].loc[thin_5_9.columns[j]].values[0].split(' [')[0]
        count+=1
        j += 3
        

thin_5_9_data['Year'] = thin_5_9_data['Year'].astype(np.int64)
#thin_5_9_data['Thinness_5-9_years'] = thin_5_9_data['Thinness_5-9_years'].astype(np.float64)

thin_5_9_data.to_pickle("Data/New/Thinness_5_9/thin_5_9_internet.pkl")

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(thin_5_9_data[thin_5_9_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            thin_5_9_other_data = thin_5_9_data[thin_5_9_data['Country'] == main_data['Country'][val]][thin_5_9_data['Year'] == main_data['Year'][val]]['Thinness_5-9_years'].values[0]
            print(thin_5_9_other_data, "pass: 1")
            main_data.at[val, 'Thinness_5-9_years'] = thin_5_9_other_data
        except IndexError:
            pass
    else:
        try:
            thin_5_9_other_data = thin_5_9_data[thin_5_9_data['Country'].str.startswith(main_data['Country'][val])][thin_5_9_data['Year'] == main_data['Year'][val]]['Thinness_5-9_years'].values[0]
            print(thin_5_9_other_data, "pass: 2")
            main_data.at[val, 'Thinness_5-9_years'] = thin_5_9_other_data
        except IndexError:
            try:
                thin_5_9_other_data = thin_5_9_data[thin_5_9_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][thin_5_9_data['Year'] == main_data['Year'][val]]['Thinness_5-9_years'].values[0]
                print(thin_5_9_other_data, "pass: 3")
                main_data.at[val, 'Thinness_5-9_years'] = thin_5_9_other_data
            except IndexError:
                pass

main_data[['Country','Year','Thinness_5-9_years']].isnull().sum()

## STATUS OF EACH COUNTRY

In [ ]:
status_data = pd.read_csv("Data/New/Status/Status_Of_Country.csv")

status_data.rename(columns = {status_data.columns[0]: "ID"}, inplace=True)
status_data.columns

status_data.head()

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(status_data[status_data['Economy'] == main_data['Country'][val]].values) != 0:
        try:
            status_other_data = status_data[status_data['Economy'] == main_data['Country'][val]]["Income group"].values[0]
            print(status_other_data, "pass: 1")
            main_data.at[val, 'Status'] = status_other_data
        except IndexError:
            pass
    else:
        try:
            status_other_data = status_data[status_data['Economy'].str.startswith(main_data['Country'][val])]["Income group"].values[0]
            print(status_other_data, "pass: 2")
            main_data.at[val, 'Status'] = status_other_data
        except IndexError:
            try:
                status_other_data = status_data[status_data['Economy'].str.startswith(main_data['Country'][val].split(',')[0])]["Income group"].values[0]
                print(status_other_data, "pass: 3")
                main_data.at[val, 'Status'] = status_other_data
            except IndexError:
                pass

#main_data.sample(10)

## POLIO

In [ ]:
polio = pd.read_csv("Data/New/Polio/Polio_Internet.csv")

polio.head()

count = 0
polio_data = pd.DataFrame(columns=['Country', 'Year', 'Polio'])
for i in range(0, len(polio['Country'].unique())):
    for j in range(1, len(polio.columns)):
        #print(polio.loc[i,'Country'], end='')
        polio_data.at[count, 'Country'] = polio.loc[i,'Country']
        polio_data.at[count, 'Year'] = polio.columns[j]
       # print(polio.columns[j])
        polio_data.at[count, 'Polio'] = polio[polio['Country'] == polio['Country'][i]].T[1:].loc[polio.columns[j]].values[0]
        count+=1

polio_data['Year'] = polio_data['Year'].astype(np.int64)
polio_data.info()

polio_data.to_pickle("Data/New/Polio/Polio_Internet.pkl")

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(polio_data[polio_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            polio_other_data = polio_data[polio_data['Country'] == main_data['Country'][val]][polio_data['Year'] == main_data['Year'][val]]['Polio'].values[0]
            if regex_double_nums.match(str(polio_other_data)):
                polio_other_data = polio_other_data.split(' ')[0]
                print('Regex Matched', end=' ')
            print(polio_other_data, "pass: 1")
            main_data.at[val, 'Polio'] = polio_other_data
        except IndexError:
            pass
    else:
        try:
            polio_other_data = polio_data[polio_data['Country'].str.startswith(main_data['Country'][val])][polio_data['Year'] == main_data['Year'][val]]['Polio'].values[0]
            if regex_double_nums.match(str(polio_other_data)):
                polio_other_data = polio_other_data.split(' ')[0]
                print('Regex Matched', end=' ')
            print(polio_other_data, "pass: 2")
            main_data.at[val, 'Polio'] = polio_other_data
        except IndexError:
            try:
                polio_other_data = polio_data[polio_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][polio_data['Year'] == main_data['Year'][val]]['Polio'].values[0]
                if regex_double_nums.match(str(polio_other_data)):
                    polio_other_data = polio_other_data.split(' ')[0]
                    print('Regex Matched', end=' ')
                print(polio_other_data, "pass: 3")
                main_data.at[val, 'Polio'] = polio_other_data
            except IndexError:
                pass
         

#main_data.iloc[[2925]]

## Under Five Deaths

In [ ]:
under_5_data = pd.read_csv("Data/New/Under_5_Deaths/Under_5_Internet.csv")

under_5_data.rename(columns = {under_5_data.columns[2]: "Under-Five_Deaths"}, inplace=True)
under_5_data.columns

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(under_5_data[under_5_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            under_5_other_data = under_5_data[under_5_data['Country'] == main_data['Country'][val]][under_5_data['Year'] == main_data['Year'][val]]['Under-Five_Deaths'].values[0]
            if (under_5_other_data == "&lt;1"):
                random.seed(val)
                under_5_other_data = random.uniform(0, 1)
            print(under_5_other_data, "pass: 1")
            main_data.at[val, 'Under-Five_Deaths'] = under_5_other_data
        except IndexError:
            pass
    else:
        try:
            under_5_other_data = under_5_data[under_5_data['Country'].str.startswith(main_data['Country'][val])][under_5_data['Year'] == main_data['Year'][val]]['Under-Five_Deaths'].values[0]
            if (under_5_other_data == "&lt;1"):
                random.seed(val)
                under_5_other_data = random.uniform(0, 1)
            print(under_5_other_data, "pass: 2")
            main_data.at[val, 'Under-Five_Deaths'] = under_5_other_data
        except IndexError:
            try:
                under_5_other_data = under_5_data[under_5_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][under_5_data['Year'] == main_data['Year'][val]]['Under-Five_Deaths'].values[0]
                if (under_5_other_data == "&lt;1"):
                    random.seed(val)
                    under_5_other_data = random.uniform(0, 1)
                print(under_5_other_data, "pass: 3")
                main_data.at[val, 'Under-Five_Deaths'] = under_5_other_data
            except IndexError:
                pass
         

main_data[['Country','Year','Under-Five_Deaths']].head()

under_5_data.head()

## Infant Deaths

In [ ]:
infant_death_data = pd.read_csv("Data/New/Infant_Deaths/Infant_Deaths_Internet.csv")

infant_death_data.rename(columns = {infant_death_data.columns[3]: "Infant_Deaths"}, inplace=True)
infant_death_data.columns

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(infant_death_data[infant_death_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            infant_death_other_data = infant_death_data[infant_death_data['Country'] == main_data['Country'][val]][infant_death_data['Year'] == main_data['Year'][val]]['Infant_Deaths'].values[0]
            if (infant_death_other_data == "&lt;1"):
                random.seed(val)
                infant_death_other_data = random.uniform(0, 1)
            print(infant_death_other_data, "pass: 1")
            main_data.at[val, 'Infant_Deaths'] = infant_death_other_data
        except IndexError:
            pass
    else:
        try:
            infant_death_other_data = infant_death_data[infant_death_data['Country'].str.startswith(main_data['Country'][val])][infant_death_data['Year'] == main_data['Year'][val]]['Infant_Deaths'].values[0]
            if (infant_death_other_data == "&lt;1"):
                random.seed(val)
                infant_death_other_data = random.uniform(0, 1)
            print(infant_death_other_data, "pass: 2")
            main_data.at[val, 'Infant_Deaths'] = infant_death_other_data
        except IndexError:
            try:
                infant_death_other_data = infant_death_data[infant_death_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][infant_death_data['Year'] == main_data['Year'][val]]['Infant_Deaths'].values[0]
                if (infant_death_other_data == "&lt;1"):
                    random.seed(val)
                    infant_death_other_data = random.uniform(0, 1)
                print(infant_death_other_data, "pass: 3")
                main_data.at[val, 'Infant_Deaths'] = infant_death_other_data
            except IndexError:
                pass
         

## ADULT MORTALITY

In [ ]:
adult_mor_data = pd.read_csv("Data/New/Adult_Mortality/Ad_Mor_Internet.csv")

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(adult_mor_data[adult_mor_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            adult_mor_other_data = adult_mor_data[adult_mor_data['Country'] == main_data['Country'][val]][adult_mor_data['Year'] == main_data['Year'][val]]['Adult_Mortality'].values[0]
            print(adult_mor_other_data, "pass: 1")
            main_data.at[val, 'Adult_Mortality'] = adult_mor_other_data
        except IndexError:
            pass
    else:
        try:
            adult_mor_other_data = adult_mor_data[adult_mor_data['Country'].str.startswith(main_data['Country'][val])][adult_mor_data['Year'] == main_data['Year'][val]]['Adult_Mortality'].values[0]
            print(adult_mor_other_data, "pass: 2")
            main_data.at[val, 'Adult_Mortality'] = adult_mor_other_data
        except IndexError:
            try:
                adult_mor_other_data = adult_mor_data[adult_mor_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][adult_mor_data['Year'] == main_data['Year'][val]]['Adult_Mortality'].values[0]
                print(adult_mor_other_data, "pass: 3")
                main_data.at[val, 'Adult_Mortality'] = adult_mor_other_data
            except IndexError:
                pass

#adult_mor_data.head()

#main_data.head()

## Life Expectancy

In [ ]:
life_exp_data = pd.read_csv("Data/New/Life_Expectancy/life_exp_internet.csv")

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(life_exp_data[life_exp_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            life_exp_other_data = life_exp_data[life_exp_data['Country'] == main_data['Country'][val]][life_exp_data['Year'] == main_data['Year'][val]]['Life_Expectancy'].values[0]
            if regex_double_nums.match(life_exp_other_data):
                life_exp_other_data = life_exp_other_data.split(' ')[0]
                print('Regex Matched', end=' ')
            print(life_exp_other_data, "pass: 1")
            main_data.at[val, 'Life_Expectancy'] = life_exp_other_data
        except IndexError:
            pass
    else:
        try:
            life_exp_other_data = life_exp_data[life_exp_data['Country'].str.startswith(main_data['Country'][val])][life_exp_data['Year'] == main_data['Year'][val]]['Life_Expectancy'].values[0]
            if regex_double_nums.match(life_exp_other_data):
                life_exp_other_data = life_exp_other_data.split(' ')[0]
                print('Regex Matched', end=' ')
            print(life_exp_other_data, "pass: 2")
            main_data.at[val, 'Life_Expectancy'] = life_exp_other_data
        except IndexError:
            try:
                life_exp_other_data = life_exp_data[life_exp_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][life_exp_data['Year'] == main_data['Year'][val]]['Life_Expectancy'].values[0]
                if regex_double_nums.match(life_exp_other_data):
                    life_exp_other_data = life_exp_other_data.split(' ')[0]
                    print('Regex Matched', end=' ')
                print(life_exp_other_data, "pass: 3")
                main_data.at[val, 'Life_Expectancy'] = life_exp_other_data
            except IndexError:
                pass

#main_data[['Country','Year','Life_Expectancy']].head()

## Percentage Expenditure

In [ ]:
per_exp = pd.read_csv("Data/New/Percent_Expenditure/Percent_Exp_Internet.csv")

count = 0
per_exp_data = pd.DataFrame(columns=['Country', 'Year', 'Percentage_Expenditure'])
for i in range(0, len(per_exp['Country'].unique())):
    for j in range(1, len(per_exp.columns)):
        #print(per_exp.loc[i,'Country'], end='')
        per_exp_data.at[count, 'Country'] = per_exp.loc[i,'Country']
        per_exp_data.at[count, 'Year'] = per_exp.columns[j]
       # print(per_exp.columns[j])
        per_exp_data.at[count, 'Percentage_Expenditure'] = per_exp[per_exp['Country'] == per_exp['Country'][i]].T[1:].loc[per_exp.columns[j]].values[0]
        count+=1

per_exp_data['Year'] = per_exp_data['Year'].astype(np.int64)

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(per_exp_data[per_exp_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            per_exp_other_data = per_exp_data[per_exp_data['Country'] == main_data['Country'][val]][per_exp_data['Year'] == main_data['Year'][val]]['Percentage_Expenditure'].values[0]
            print(per_exp_other_data, "pass: 1")
            main_data.at[val, 'Percentage_Expenditure'] = per_exp_other_data
        except IndexError:
            pass
    else:
        try:
            per_exp_other_data = per_exp_data[per_exp_data['Country'].str.startswith(main_data['Country'][val])][per_exp_data['Year'] == main_data['Year'][val]]['Percentage_Expenditure'].values[0]
            print(per_exp_other_data, "pass: 2")
            main_data.at[val, 'Percentage_Expenditure'] = per_exp_other_data
        except IndexError:
            try:
                per_exp_other_data = per_exp_data[per_exp_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][per_exp_data['Year'] == main_data['Year'][val]]['Percentage_Expenditure'].values[0]
                print(per_exp_other_data, "pass: 3")
                main_data.at[val, 'Percentage_Expenditure'] = per_exp_other_data
            except IndexError:
                pass

#main_data[['Country','Year','Percentage_Expenditure']].head(10)

#per_exp_data.head(10)

## Alcohol

In [ ]:
alcohol_2000_2009 = pd.read_csv("Data/New/Alcohol/Alcohol_2000_2009.csv")

#alcohol_2000_2009.tail(10)

count = 0
count_country = 0
alcohol_2000_2009_data = pd.DataFrame(columns=['Country', 'Year', 'Alcohol'])
for i in range(0, len(alcohol_2000_2009['Country'].unique())+1):
    for j in range(2, len(alcohol_2000_2009.columns)):
        print(alcohol_2000_2009.loc[count_country,'Country'], end=' ')
        alcohol_2000_2009_data.at[count, 'Country'] = alcohol_2000_2009.loc[count_country,'Country']
        alcohol_2000_2009_data.at[count, 'Year'] = alcohol_2000_2009.columns[j]
        print(i, count_country)
        alcohol_2000_2009_data.at[count, 'Alcohol'] = alcohol_2000_2009[alcohol_2000_2009['Country'] == alcohol_2000_2009['Country'][count_country]].T[1:].loc[alcohol_2000_2009.columns[j]].values[0]
        count+=1
    count_country += 5

#alcohol_2000_2009_data[alcohol_2000_2009_data["Country"] == "Zimbabwe"]

alcohol_2010_X = pd.read_csv("Data/New/Alcohol/Alcohol_2010_x.csv")

count = 0
count_country = 0
alcohol_2010_X_data = pd.DataFrame(columns=['Country', 'Year', 'Alcohol'])
for i in range(0, len(alcohol_2010_X['Country'].unique())+1):
    for j in range(2, len(alcohol_2010_X.columns)):
        print(alcohol_2010_X.loc[count_country,'Country'], end=' ')
        alcohol_2010_X_data.at[count, 'Country'] = alcohol_2010_X.loc[count_country,'Country']
        alcohol_2010_X_data.at[count, 'Year'] = alcohol_2010_X.columns[j]
        print(i, count_country)
        alcohol_2010_X_data.at[count, 'Alcohol'] = alcohol_2010_X[alcohol_2010_X['Country'] == alcohol_2010_X['Country'][count_country]].T[1:].loc[alcohol_2010_X.columns[j]].values[0]
        count+=1
    count_country += 5

alcohol_2000_2009_data['Year'] = alcohol_2000_2009_data['Year'].astype(np.int64)
alcohol_2010_X_data['Year'] = alcohol_2010_X_data['Year'].astype(np.int64)

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(per_exp_data[per_exp_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            per_exp_other_data = per_exp_data[per_exp_data['Country'] == main_data['Country'][val]][per_exp_data['Year'] == main_data['Year'][val]]['Percentage_Expenditure'].values[0]
            print(per_exp_other_data, "pass: 1")
            main_data.at[val, 'Percentage_Expenditure'] = per_exp_other_data
        except IndexError:
            pass
    else:
        try:
            per_exp_other_data = per_exp_data[per_exp_data['Country'].str.startswith(main_data['Country'][val])][per_exp_data['Year'] == main_data['Year'][val]]['Percentage_Expenditure'].values[0]
            print(per_exp_other_data, "pass: 2")
            main_data.at[val, 'Percentage_Expenditure'] = per_exp_other_data
        except IndexError:
            try:
                per_exp_other_data = per_exp_data[per_exp_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][per_exp_data['Year'] == main_data['Year'][val]]['Percentage_Expenditure'].values[0]
                print(per_exp_other_data, "pass: 3")
                main_data.at[val, 'Percentage_Expenditure'] = per_exp_other_data
            except IndexError:
                pass

alcohol_appended = alcohol_2010_X_data.append(alcohol_2000_2009_data)

alcohol_data = alcohol_appended.reset_index(drop=True)

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(alcohol_data[alcohol_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            alcohol_other_data = alcohol_data[alcohol_data['Country'] == main_data['Country'][val]][alcohol_data['Year'] == main_data['Year'][val]]['Alcohol'].values[0]
            print(alcohol_other_data, "pass: 1")
            main_data.at[val, 'Alcohol'] = alcohol_other_data
        except IndexError:
            pass
    else:
        try:
            alcohol_other_data = alcohol_data[alcohol_data['Country'].str.startswith(main_data['Country'][val])][alcohol_data['Year'] == main_data['Year'][val]]['Alcohol'].values[0]
            print(alcohol_other_data, "pass: 2")
            main_data.at[val, 'Alcohol'] = alcohol_other_data
        except IndexError:
            try:
                alcohol_other_data = alcohol_data[alcohol_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][alcohol_data['Year'] == main_data['Year'][val]]['Alcohol'].values[0]
                print(alcohol_other_data, "pass: 3")
                main_data.at[val, 'Alcohol'] = alcohol_other_data
            except IndexError:
                pass

## BMI 5-19 Years

In [ ]:
bmi_5_19 = pd.read_csv("Data/New/BMI_5-19/BMI_5-19_Internet.csv")

bmi_5_19.rename(columns = {bmi_5_19.columns[0]: "Country"}, inplace=True)
bmi_5_19.head()

count = 0
bmi_5_19_data = pd.DataFrame(columns=['Country', 'Year', 'BMI_5-19_years'])
for i in range(1, len(bmi_5_19['Country'].unique())):
    j = 1
    while j < len(bmi_5_19.columns):
        #print(bmi_5_19.loc[i,'Country'], end='')
        bmi_5_19_data.at[count, 'Country'] = bmi_5_19.loc[i,'Country']
        bmi_5_19_data.at[count, 'Year'] = bmi_5_19.columns[j]
       # print(bmi_5_19.columns[j])
        bmi_5_19_data.at[count, 'BMI_5-19_years'] = bmi_5_19[bmi_5_19['Country'] == bmi_5_19['Country'][i]].T[1:].loc[bmi_5_19.columns[j]].values[0].split(' [')[0]
        count+=1
        j += 3

bmi_5_19_data.head()

bmi_5_19_data['Year'] = bmi_5_19_data['Year'].astype(np.int64)

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(bmi_5_19_data[bmi_5_19_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            bmi_5_19_other_data = bmi_5_19_data[bmi_5_19_data['Country'] == main_data['Country'][val]][bmi_5_19_data['Year'] == main_data['Year'][val]]['BMI_5-19_years'].values[0]
            print(bmi_5_19_other_data, "pass: 1")
            main_data.at[val, 'BMI_5-19_years'] = bmi_5_19_other_data
        except IndexError:
            pass
    else:
        try:
            bmi_5_19_other_data = bmi_5_19_data[bmi_5_19_data['Country'].str.startswith(main_data['Country'][val])][bmi_5_19_data['Year'] == main_data['Year'][val]]['BMI_5-19_years'].values[0]
            print(bmi_5_19_other_data, "pass: 2")
            main_data.at[val, 'BMI_5-19_years'] = bmi_5_19_other_data
        except IndexError:
            try:
                bmi_5_19_other_data = bmi_5_19_data[bmi_5_19_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][bmi_5_19_data['Year'] == main_data['Year'][val]]['BMI_5-19_years'].values[0]
                print(bmi_5_19_other_data, "pass: 3")
                main_data.at[val, 'BMI_5-19_years'] = bmi_5_19_other_data
            except IndexError:
                pass

## BMI 18+ Years

In [ ]:
bmi_18 = pd.read_csv("Data/New/BMI_18+/BMI_18+_Internet.csv")

bmi_18.rename(columns = {bmi_18.columns[0]: "Country"}, inplace=True)
bmi_18.head()

count = 0
bmi_18_data = pd.DataFrame(columns=['Country', 'Year', 'BMI_18+_years'])
for i in range(1, len(bmi_18['Country'].unique())):
    j = 1
    while j < len(bmi_18.columns):
        #print(bmi_18.loc[i,'Country'], end='')
        bmi_18_data.at[count, 'Country'] = bmi_18.loc[i,'Country']
        bmi_18_data.at[count, 'Year'] = bmi_18.columns[j]
       # print(bmi_18.columns[j])
        bmi_18_data.at[count, 'BMI_18+_years'] = bmi_18[bmi_18['Country'] == bmi_18['Country'][i]].T[1:].loc[bmi_18.columns[j]].values[0].split(' [')[0]
        count+=1
        j += 3

bmi_18_data.head()

bmi_18_data['Year'] = bmi_18_data['Year'].astype(np.int64)

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(bmi_18_data[bmi_18_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            bmi_18_other_data = bmi_18_data[bmi_18_data['Country'] == main_data['Country'][val]][bmi_18_data['Year'] == main_data['Year'][val]]['BMI_18+_years'].values[0]
            print(bmi_18_other_data, "pass: 1")
            main_data.at[val, 'BMI_18+_years'] = bmi_18_other_data
        except IndexError:
            pass
    else:
        try:
            bmi_18_other_data = bmi_18_data[bmi_18_data['Country'].str.startswith(main_data['Country'][val])][bmi_18_data['Year'] == main_data['Year'][val]]['BMI_18+_years'].values[0]
            print(bmi_18_other_data, "pass: 2")
            main_data.at[val, 'BMI_18+_years'] = bmi_18_other_data
        except IndexError:
            try:
                bmi_18_other_data = bmi_18_data[bmi_18_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][bmi_18_data['Year'] == main_data['Year'][val]]['BMI_18+_years'].values[0]
                print(bmi_18_other_data, "pass: 3")
                main_data.at[val, 'BMI_18+_years'] = bmi_18_other_data
            except IndexError:
                pass

main_data.head()

## SCHOOLING

In [ ]:
school = pd.read_csv("Data/New/Education/Education_Internet.csv")

count = 0
school_data = pd.DataFrame(columns=['Country', 'Year', 'Schooling'])
for i in range(0, len(school['Country'].unique())):
    for j in range(1, len(school.columns)):
        #print(school.loc[i,'Country'], end='')
        # Space before a word by default in the dataset
        school_data.at[count, 'Country'] = school.loc[i,'Country'].split(' ',1)[1]
        school_data.at[count, 'Year'] = school.columns[j]
        #print(school.columns[j])
        school_data.at[count, 'Schooling'] = school[school['Country'] == school['Country'][i]].T[1:].loc[school.columns[j]].values[0]
        count+=1

school_data['Year'] = school_data['Year'].astype(np.int64)

for val in main_data['Country'].index:
    print(val,' ',end='')
    if len(school_data[school_data['Country'] == main_data['Country'][val]].values) != 0:
        try:
            school_other_data = school_data[school_data['Country'] == main_data['Country'][val]][school_data['Year'] == main_data['Year'][val]]['Schooling'].values[0]
            print(school_other_data, "pass: 1")
            main_data.at[val, 'Schooling'] = school_other_data
        except IndexError:
            pass
    else:
        try:
            school_other_data = school_data[school_data['Country'].str.startswith(main_data['Country'][val])][school_data['Year'] == main_data['Year'][val]]['Schooling'].values[0]
            print(school_other_data, "pass: 2")
            main_data.at[val, 'Schooling'] = school_other_data
        except IndexError:
            try:
                school_other_data = school_data[school_data['Country'].str.startswith(main_data['Country'][val].split(',')[0])][school_data['Year'] == main_data['Year'][val]]['Schooling'].values[0]
                print(school_other_data, "pass: 3")
                main_data.at[val, 'Schooling'] = school_other_data
            except IndexError:
                pass

#school_data[school_data['Country'] == "Zimbabwe"]

#main_data[['Country','Year','Schooling']]

## Changing "No Data" to "NaNs"

In [ ]:
# Replace "No Data" with "NaNs"
for i in range(0, len(main_data.columns)):
    for j in range(0, len(main_data)):
        value = main_data[main_data.columns[i]].tolist()[j]
        if value == "No data" or value == "No Data":
                main_data.at[j, main_data.columns[i]] = np.nan

## SAVING MAIN_DATA AS PICKLE and CSV DATASET

In [ ]:
# Final save
pd.to_pickle(main_data, "Data/New/Main_Data/Dataset.pkl")

# Final save
main_data.to_csv("Data/New/Main_Data/Dataset.csv",index=False)